In [1]:
import ee

In [2]:
import geemap

In [3]:
geemap.set_proxy(port=7890)

In [4]:
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1Adeu5BXs81vubqGgmyrcSmPz-4KpYI3svTs-vyGjPdSZVe0hy9fT5QeEGss



Successfully saved authorization token.


In [5]:
image = ee.Image('srtm90_v4')
print(image.getInfo())

{'type': 'Image', 'bands': [{'id': 'elevation', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [432000, 144000], 'crs': 'EPSG:4326', 'crs_transform': [0.000833333333333, 0, -180, 0, -0.000833333333333, 60]}], 'version': 1494271934303000.0, 'id': 'srtm90_v4', 'properties': {'system:time_start': 950227200000, 'system:time_end': 951177600000, 'system:asset_size': 18827626666}}


In [7]:
import pdb
pdb.set_trace()

--Call--
> c:\users\chl\.conda\envs\geemap_chl\lib\site-packages\ipython\core\displayhook.py(258)__call__()
    256         sys.stdout.flush()
    257 
--> 258     def __call__(self, result=None):
    259         """Printing with history cache management.
    260 



ipdb>  exit()


In [207]:
!pip install geemap
# !pip install eerepr
!pip install geopandas
import os,tarfile,sys
# from google.colab import files,drive
# drive.mount('/content/drive')
# os.chdir('/content/drive/My Drive/Output')
# sys.path.append('/content/drive/My Drive/Python_Script')
import ee
import geemap
import numpy as np
import pandas as pd
import folium
import copy
import math
from tqdm import trange,tqdm
from functools import partial
import geopandas as gpd
# geemap.set_proxy(port=10809)
ee.Authenticate()
ee.Initialize()
print('geemap version = {}\ngeemap path = {}'.format(geemap.__version__,geemap.__path__))

Enter verification code:  4/1Adeu5BW5eI8HrEZsvn7QL-pUw_glMpwzcoSkXz-6lmnECfMOkLYlZ7EzGis



Successfully saved authorization token.
geemap version = 0.25.0
geemap path = ['C:\\Users\\CHL\\.conda\\envs\\geemap_chl\\Lib\\site-packages\\geemap']


In [208]:
Glacial_lake_2015A = ee.FeatureCollection(
    'projects/ee-mrwurenzhe/assets/Glacial_lake/Wu_Asia_Southest_GL_wgs84').filter(ee.Filter.gte('GL_Area', 0.1))
    #projects/ee-mrwurenzhe/assets/Glacial_lake/Checkout_polygen

# 计算geometry、质心点、最小包络矩形
Geo_ext = lambda feature: feature.set({
    'Geo': feature.geometry(),
    'Centroid': feature.geometry().centroid(),
    'Rectangle': feature.geometry().bounds()
})

Centrid_set = lambda feature: feature.setGeometry(feature.geometry().centroid())
Rectangle_set = lambda feature: feature.setGeometry(feature.geometry().bounds())
Glacial_lake_2015C = Glacial_lake_2015A.map(Geo_ext).map(Centrid_set)  # 添加属性,修改geometry,计算质心
Glacial_lake_2015R = Glacial_lake_2015A.map(Rectangle_set)       # 计算最小包络矩形

#抽取属性作为list
Glacial_lake_2015A_GeoList = ee.List(Glacial_lake_2015C.reduceColumns(ee.Reducer.toList(), ['Geo']).get('list'))
Glacial_lake_2015C_CentriodList = ee.List(Glacial_lake_2015C.reduceColumns(ee.Reducer.toList(),['Centroid']).get('list'))
Glacial_lake_2015R_RectangleList = ee.List(Glacial_lake_2015C.reduceColumns(ee.Reducer.toList(),['Rectangle']).get('list'))
Num_list = Glacial_lake_2015C_CentriodList.size().getInfo()

In [209]:
START_DATE  = ee.Date('2019-08-01')
END_DATE   = ee.Date('2019-08-30')
DEM = ee.Image('USGS/SRTMGL1_003')
models = ['volume', 'surface', None]
Buffer = 0
Model = models[0]
I = 1143
scale = 30

AOI = ee.Feature.geometry(Glacial_lake_2015R_RectangleList.get(I))
s1_col = (ee.ImageCollection("COPERNICUS/S1_GRD")
          .filter(ee.Filter.eq('instrumentMode', 'IW'))
            .filterBounds(AOI)
            .filterDate(START_DATE, END_DATE))
AOI_buffer = AOI.buffer(distance=2000).bounds()

In [210]:
s1_descending = s1_col.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')).first()
s1_ascending = s1_col.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')).first()

In [211]:
image = s1_descending
orbitProperties_pass = image.get('orbitProperties_pass').getInfo()
elevation = DEM
# get the image geometry and projection
geom = image.geometry()
proj = image.select(1).projection()

def getASCCorners(image, AOI_buffer):
    # 真实方位角
    coords = ee.Array(image.geometry().coordinates().get(0)).transpose()
    crdLons = ee.List(coords.toList().get(0))
    crdLats = ee.List(coords.toList().get(1))
    minLon = crdLons.sort().get(0)
    maxLon = crdLons.sort().get(-1)
    minLat = crdLats.sort().get(0)
    maxLat = crdLats.sort().get(-1)
    azimuth = (ee.Number(crdLons.get(crdLats.indexOf(minLat))).subtract(minLon).atan2(ee.Number(crdLats.get(crdLons.indexOf(minLon))).subtract(minLat))
               .multiply(180.0 / math.pi))

    if orbitProperties_pass == 'ASCENDING':
        azimuth = azimuth.add(270.0)
        rotationFromNorth = azimuth.subtract(360.0)
    elif orbitProperties_pass == 'DESCENDING':
        azimuth = azimuth.add(180.0)
        rotationFromNorth = azimuth.subtract(180.0)
    else:
        raise TypeError
    azimuthEdge = (ee.Feature(ee.Geometry.LineString([crdLons.get(crdLats.indexOf(minLat)), minLat, minLon,
                              crdLats.get(crdLons.indexOf(minLon))]), {'azimuth': azimuth}).copyProperties(image))

    coords = ee.Array(image.clip(
        AOI_buffer).geometry().coordinates().get(0)).transpose()
    crdLons = ee.List(coords.toList().get(0))
    crdLats = ee.List(coords.toList().get(1))
    minLon = crdLons.sort().get(0)
    maxLon = crdLons.sort().get(-1)
    minLat = crdLats.sort().get(0)
    maxLat = crdLats.sort().get(-1)
    azimuth = (ee.Number(crdLons.get(crdLats.indexOf(minLat))).subtract(minLon).atan2(ee.Number(crdLats.get(crdLons.indexOf(minLon))).subtract(minLat))
               .multiply(180.0 / math.pi))
    if orbitProperties_pass == 'ASCENDING':
        # 左上角
        startpoint = ee.List([minLon, maxLat])
        # 右下角
        endpoint = ee.List([maxLon, minLat])
    elif orbitProperties_pass == 'DESCENDING':
        # 右上角
        startpoint = ee.List([maxLon, maxLat])
        # 左下角
        endpoint = ee.List([minLon, minLat])

    coordinates_dict = {'crdLons': crdLons, 'crdLats': crdLats,
                        'minLon': minLon, 'maxLon': maxLon, 'minLat': minLat, 'maxLat': maxLat}

    return azimuthEdge, rotationFromNorth, startpoint, endpoint, coordinates_dict
Angle_aspect = ee.Terrain.aspect(image.select('angle'))
# s1_azimuth_across = Angle_aspect.reduceRegion(ee.Reducer.mean(), geom, 1000).get('aspect')

azimuthEdge, rotationFromNorth, startpoint, endpoint, coordinates_dict = getASCCorners(image, AOI_buffer)
Heading = azimuthEdge.get('azimuth')
Heading_Rad = ee.Image.constant(Heading).multiply(np.pi / 180)

s1_azimuth_across = ee.Number(Heading).subtract(90.0)
theta_iRad = image.select('angle').multiply(np.pi / 180)  # 地面入射角度转为弧度
phi_iRad = ee.Image.constant(s1_azimuth_across).multiply(np.pi / 180)  # 方位角转弧度

alpha_sRad = ee.Terrain.slope(elevation).select('slope').multiply(
    np.pi / 180).setDefaultProjection(proj).clip(geom)          # 坡度(与地面夹角)
phi_sRad = ee.Terrain.aspect(elevation).select('aspect').multiply(
    np.pi / 180).setDefaultProjection(proj).clip(geom)          # 坡向角，(坡度陡峭度)坡与正北方向夹角(陡峭度)，从正北方向起算，顺时针计算角度

height = elevation.setDefaultProjection(proj).clip(geom)

phi_rRad = phi_iRad.subtract(phi_sRad)     # (飞行方向角度-坡度陡峭度)飞行方向与坡向之间的夹角
# 分解坡度，在水平方向和垂直方向进行分解，为固定公式，cos对应水平分解，sin对应垂直分解
alpha_rRad = (alpha_sRad.tan().multiply(phi_rRad.cos())).atan()     # 距离向分解
alpha_azRad = (alpha_sRad.tan().multiply(phi_rRad.sin())).atan()     # 方位向分解

theta_liaRad = (alpha_azRad.cos().multiply(
    (theta_iRad.subtract(alpha_rRad)).cos())).acos()               # LIA
theta_liaDeg = theta_liaRad.multiply(180 / np.pi)                # LIA转弧度



# ------------------------------RS几何畸变区域--------------------------------- 同戴可人：高山峡谷区滑坡灾害隐患InSAR早期识别
layover = alpha_rRad.gt(theta_iRad).rename('layover')
ninetyRad = ee.Image.constant(90).multiply(np.pi / 180)
shadow = alpha_rRad.lt(ee.Image.constant(-1).multiply(ninetyRad.subtract(theta_iRad))).rename('shadow')

# # ------------------------------IJRS几何畸变区域-------------------------------
# layover = alpha_rRad.gt(theta_iRad).rename('layover')
# shadow = theta_liaDeg.gt(ee.Image.constant(85).multiply(np.pi / 180)).rename('shadow')

# # ------------------------------武大学报几何畸变区域---------------------------
# layover = theta_liaDeg.lt(ee.Image.constant(0).multiply(np.pi / 180)).rename('layover')
# shadow = theta_liaDeg.gt(ee.Image.constant(90).multiply(np.pi / 180)).rename('shadow')

# RINDEX，暂时无用
# if orbitProperties_pass == 'ASCENDING':
#   A = phi_sRad.subtract(Heading_Rad)
# elif orbitProperties_pass == 'DESCENDING':
#   A = phi_sRad.add(Heading_Rad).add(np.pi)
# R_Index = theta_iRad.subtract(alpha_sRad.multiply(A.sin()))
# layover =


def rgbmask(layover, shadow):
    r = ee.Image.constant(0).select([0], ['red'])
    g = ee.Image.constant(0).select([0], ['green'])
    b = ee.Image.constant(0).select([0], ['blue'])

    r = r.where(layover, 255)
    g = g.where(shadow, 255)

    return ee.Image.cat([r, g, b]).byte().updateMask(image.mask())

# combine layover and shadow,因为shadow和layover都是0
no_data_mask = layover.And(shadow).rename('no_data_mask')
no_data_maskrgb = rgbmask(layover, shadow)

def Eq_pixels(x): return ee.Image.constant(0).where(x, x).updateMask(x.mask())

image2 = (Eq_pixels(image.select('VV')).rename('VV_sigma0').addBands(Eq_pixels(image.select('VH')).rename('VH_sigma0'))
          .addBands(Eq_pixels(image.select('angle')).rename('incAngle')).addBands(Eq_pixels(layover).rename('layover'))
          .addBands(Eq_pixels(shadow).rename('shadow'))
          .addBands(Eq_pixels(no_data_mask).rename('no_data_mask'))
          .addBands(no_data_maskrgb).addBands(Eq_pixels(height).rename('height')))

cal_image = (image2.addBands(ee.Image.pixelCoordinates(proj)).addBands(ee.Image.pixelLonLat())
             .reproject(crs=proj, scale=scale).updateMask(image2.select('VV_sigma0').mask()).clip(AOI_buffer))
height = cal_image.select('height')
layover = cal_image.select('layover')
shadow = cal_image.select('shadow')
no_data_mask = cal_image.select('no_data_mask')
no_data_maskrgb = cal_image.select('no_data_maskrgb')

print("Corrected across-range-look direction", s1_azimuth_across.getInfo())
# print("True azimuth across-range-look direction", ee.Number(Heading).subtract(90.0).getInfo())

Corrected across-range-look direction 98.9867776867089


In [212]:
Map = geemap.Map(basemap='HYBRID')
Map.centerObject(AOI, zoom=14)
Map.addLayer(AOI, {'color': 'green'}, 'Center')
Auxiliarylines = ee.Geometry.LineString([startpoint,endpoint])
Map.addLayer(Auxiliarylines, {}, 'Auxiliarylines')
Map.addLayer(cal_image.select('VV_sigma0'),{'min':-40,'max':5}, 'VV_sigma0')
# Map.addLayer(cal_image.select('x'),{},'x')
Map.addLayer(cal_image.select('red','green','blue'), {'min':0,'max':255}, 'no_data_maskrgb')
Map # 显示地图窗口

Map(center=[30.17633989163365, 94.28168536055013], controls=(WidgetControl(options=['position', 'transparent_b…

In [213]:
# 1、选一根线，逐点计算，获取梯度方向，构建梯度向量
# 2、根据已有的方位向量，构建y=kx+c的方程，使用该方程沿对角线像素进行遍历，计算路径上的被动几何畸变
# 将方位角转化为斜率
# 用于分析栅格像元
def reduce_tolist(each): return ee.Image(each).reduceRegion(
    reducer=ee.Reducer.toList(), geometry=image.geometry(), scale=scale, maxPixels=1e13)

def angle2slope(angle):
    if type(angle) == ee.ee_number.Number:
        angle = angle.getInfo()
    if 0 < angle <= 90 or 180 < angle <= 270:
        if 180 < angle <= 270:
            angle = 90-(angle - 180)
        arc = angle/180*math.pi
        slop = math.tan(arc)
    elif 90 < angle <= 180 or 270 < angle <= 360:
        if 90 < angle <= 180:
            angle = angle - 90
        elif 270 < angle <= 360:
            angle = angle - 270
        arc = angle/180*math.pi
        slop = -math.tan(arc)
    return slop

# 计算斜率
K = angle2slope(s1_azimuth_across)

# 过Auxiliarylines中的点，从最小经度到最大经度
Max_Lon = coordinates_dict['maxLon'].getInfo()
Min_Lon = coordinates_dict['minLon'].getInfo()
AuxiliaryPoints = reduce_tolist(cal_image.select(
    [ 'longitude', 'latitude']).clip(Auxiliarylines)).getInfo()

#获取辅助线上的所有点数据
Aux_lon = np.array(AuxiliaryPoints['longitude'])
Aux_lon, indices_Aux_lon = np.unique(Aux_lon, return_index=True)
Aux_lat = np.array(AuxiliaryPoints['latitude'])[indices_Aux_lon]

Templist = []
for X, Y in zip(Aux_lon, Aux_lat):
    C = Y - K*X
    Min_Lon_Y = K*Min_Lon + C
    Max_lon_Y = K*Max_Lon + C
    Templist.append(ee.Feature(ee.Geometry.LineString(
        [Min_Lon, Min_Lon_Y, Max_Lon, Max_lon_Y])))
SightLines = ee.FeatureCollection(Templist)
Map.addLayer(SightLines, {'color': 'white'}, 'test')

In [214]:
K

-0.15814788698105856

In [218]:
from scipy.signal import argrelextrema
from tqdm import tqdm

line_points_list = []
# LPassive_layover_linList = []
# RPassive_layover_linList = []
# Shadow_linList = []

for eachLine in tqdm(Templist):
  # 求与辅助线相交的像素值,求取极值点位置
  LineImg = cal_image.select(
      ['height', 'layover', 'shadow', 'incAngle', 'x', 'y', 'longitude', 'latitude']).clip(eachLine)
  LineImg_point = LineImg.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=image.geometry(),
      scale=30,
      maxPixels=1e13)
  line_points_list.append(LineImg_point)
list_of_dicts = ee.List(line_points_list).getInfo() # 需转换数据到本地，严重耗时
# 分别为右侧被动叠掩点、左侧被动叠掩点和被动阴影点经纬度列表
r_lon_sum=[]
r_lat_sum=[]
l_lon_sum=[]
l_lat_sum=[]
sh_lon_sum=[]
sh_lat_sum=[]
# 逐视线检索被动畸变
for PointDict in tqdm(list_of_dicts):
    if orbitProperties_pass == 'ASCENDING':
        order = np.argsort(PointDict['x'])
    elif orbitProperties_pass == 'DESCENDING':
        order = np.argsort(PointDict['x'])[::-1]
    PointDict = {k: np.array(v)[order] for k, v in PointDict.items()} #将视线上的像元按照x，从小到大排序
    PointDict['x'], PointDict['y'] = PointDict['x'] * \
        10, PointDict['y'] * 10  # 像素行列10m分辨率，由proj得
    EasyIndex = lambda Data, Index, *Keys: [Data[key][Index] for key in Keys]
    # 被动阴影像元识别
    if len(np.where(PointDict['shadow']==1)[0])!=0:
        # 利用grad_fn函数识别出主动阴影起始索引和结束索引
        shadow_grad_fn=lambda shadow_array: [shadow_array[i+1]-shadow_array[i] for i in range(len(shadow_array)-1)]
        PointDict_shadow_padding=np.insert(PointDict['shadow'],0,0)
        shadow_grad=np.array(shadow_grad_fn(PointDict_shadow_padding))
        # 防止PointDict['shadow']最后是以1结尾，导致-1比1个数少一个，且1结尾没有检测被动阴影必要，所以直接去掉
        if PointDict_shadow_padding[-1]==1:
            shadow_start_index=np.where(shadow_grad==1)[0][:-1]
        else:
            shadow_start_index=np.where(shadow_grad==1)[0]
        shadow_range_index=np.where(shadow_grad==-1)[0]
        # 以主动阴影的起始像元位置为参考点
        sh_h, sh_x,s_y, sh_lon, sh_lat, sh_angle = EasyIndex(
            PointDict, shadow_start_index, 'height', 'x', 'y', 'longitude', 'latitude', 'incAngle')
        # 在某一视线中，逐段主动阴影进行分析
        if len(shadow_start_index)!=0:
            for i in range(shadow_start_index.size):
                start_sh_index=shadow_start_index[i]
                range_sh_index=shadow_range_index[i]
                index_Shadow=[]
                # 确定检索像元范围
                if range_sh_index+50<len(PointDict['shadow']):
                    rangeIndex_shadow=range(range_sh_index,range_sh_index+50)
                else:
                    rangeIndex_shadow=range(range_sh_index,len(PointDict['shadow']))
                if len(rangeIndex_shadow)!=0:
                    sh_Range_h, sh_Range_x, sh_Range_y, sh_Range_lon, sh_Range_lat = EasyIndex(
                        PointDict, rangeIndex_shadow, 'height', 'x', 'y', 'longitude', 'latitude')
                    shadow_angle_iRad = np.arctan((sh_h[i]-sh_Range_h)/np.sqrt(np.square(sh_Range_x - sh_x[i]) + np.square(sh_Range_y-s_y[i])))
                    shadow_angle=shadow_angle_iRad* 180 / math.pi
                    index_Shadow=np.where(shadow_angle > (90-sh_angle[i]))[0]

                if len(index_Shadow)!=0:
                    tlon_shadow = sh_Range_lon[index_Shadow]
                    tlat_shadow= sh_Range_lat[index_Shadow]
                    for j in range(len(tlon_shadow)):
                        sh_lon_sum.append(tlon_shadow[j])
                        sh_lat_sum.append(tlat_shadow[j])

    # 左右侧被动叠掩
    if len(np.where(PointDict['layover']==1)[0])!=0:
        layover_grad_fn=lambda lay_array: [lay_array[i+1]-lay_array[i] for i in range(len(lay_array)-1)]
        PointDict_lay_padding=np.insert(PointDict['layover'],0,0)
        # PointDict_lay_padding=np.append(PointDict_lay_padding,0)
        layover_grad=np.array(layover_grad_fn(PointDict_lay_padding))
        if PointDict_lay_padding[-1]==1:
            rlayover_start_index=np.where(layover_grad==1)[0][:-1]
        else:
            rlayover_start_index=np.where(layover_grad==1)[0]
        rlayover_range_index=np.where(layover_grad==-1)[0]
        #右侧叠掩以rlayover_start_index为参考点，rlayover_range_index为像元检索起点，左侧叠掩相反
        s_h, s_x,s_y, s_lon, s_lat, s_angle = EasyIndex(
            PointDict, rlayover_start_index, 'height', 'x', 'y', 'longitude', 'latitude', 'incAngle')
        d_h, d_x,d_y, d_lon, d_lat, d_angle = EasyIndex(
            PointDict, rlayover_range_index, 'height', 'x', 'y', 'longitude', 'latitude', 'incAngle')
        # 同时进行左右俩侧像元叠掩检测
        for i in range(rlayover_start_index.size):
            start_index=rlayover_start_index[i]
            range_index=rlayover_range_index[i]
            index_Rlayover=[]
            index_Llayover=[]
            if range_index+50<len(PointDict['layover']):
                rangeIndex=range(range_index,range_index+50)
            else:
                rangeIndex=range(range_index,len(PointDict['layover']))

            if start_index-50>0:
                rangeIndex_l=range(start_index-50,start_index)
            else:
                rangeIndex_l=range(start_index)
            # 为检测出完整的被动叠掩位置，像元应同时满足其与主动叠掩起始位置和终止位置的几何关系
            if len(rangeIndex)!=0:
                r_Range_h, r_Range_x, r_Range_y, r_Range_lon, r_Range_lat = EasyIndex(
                    PointDict, rangeIndex, 'height', 'x', 'y', 'longitude', 'latitude')
                Rlay_start_iRad = np.arctan(
                    (r_Range_h-s_h[i])/np.sqrt(np.square(r_Range_x - s_x[i]) + np.square(r_Range_y-s_y[i])))
                Rlay_end_iRad = np.arctan(
                    (r_Range_h-d_h[i])/np.sqrt(np.square(r_Range_x - d_x[i]) + np.square(r_Range_y-d_y[i])))
                Rlay_start = Rlay_start_iRad * 180 / math.pi
                Rlay_end=Rlay_end_iRad * 180 / math.pi
                index_Rlayover=np.where(np.logical_and(Rlay_start>s_angle[i],Rlay_end<d_angle[i]))[0]

            if len(rangeIndex_l)!=0:
                l_Range_h, l_Range_x, l_Range_y, l_Range_lon, l_Range_lat = EasyIndex(
                    PointDict, rangeIndex_l, 'height', 'x', 'y', 'longitude', 'latitude')
                Llay_start_iRad = np.arctan(
                    (s_h[i]-l_Range_h)/np.sqrt(np.square(s_x[i]-l_Range_x) + np.square(s_y[i]-l_Range_y)))
                Llay_end_iRad = np.arctan(
                    (d_h[i]-l_Range_h)/np.sqrt(np.square(d_x[i]-l_Range_x) + np.square(d_y[i]-l_Range_y)))
                Llay_start=Llay_start_iRad*180/math.pi
                Llay_end=Llay_end_iRad*180/math.pi
                index_Llayover=np.where(np.logical_and(Llay_start<s_angle[i],Llay_end>d_angle[i]))[0]

            if len(index_Rlayover)!=0:
                tlon_RLay = r_Range_lon[index_Rlayover]
                tlat_RLay = r_Range_lat[index_Rlayover]
                for j in range(len(tlat_RLay)):
                    r_lon_sum.append(tlon_RLay[j])
                    r_lat_sum.append(tlat_RLay[j])
            if len(index_Llayover)!=0:
                tlon_LLay = l_Range_lon[index_Llayover]
                tlat_LLay = l_Range_lat[index_Llayover]
                for j in range(len(tlat_LLay)):
                    l_lon_sum.append(tlon_LLay[j])
                    l_lat_sum.append(tlat_LLay[j])


rlay_featurecollection=ee.FeatureCollection([ee.Feature(ee.Geometry.Point(x, y), {'values': 5}) for x, y in zip(r_lon_sum, r_lat_sum)])
shadow_featurecollection=ee.FeatureCollection([ee.Feature(ee.Geometry.Point(x, y), {'values': 3}) for x, y in zip(sh_lon_sum, sh_lat_sum)])
llay_featurecollection=ee.FeatureCollection([ee.Feature(ee.Geometry.Point(x, y), {'values': 4}) for x, y in zip(l_lon_sum, l_lat_sum)])

100%|███████████████████████████████████████████████████████████████████████████████| 480/480 [00:00<00:00, 657.27it/s]


In [140]:
rlay_featurecollection2=ee.FeatureCollection([ee.Feature(ee.Geometry.Point(x, y), {'values': 5}) for x, y in zip(lon_sum[:4], lat_sum[:4])])
rlay_image2=rlay_featurecollection2.reduceToImage(['values'], ee.Reducer.mean())

In [216]:
def Line_Correct_test(cal_image,AOI_buffer,Templist, orbitProperties_pass, proj, scale:int):
    line_points_list = []
    # LPassive_layover_linList = []
    # RPassive_layover_linList = []
    # Shadow_linList = []

    for eachLine in tqdm(Templist):
        # 求与辅助线相交的像素值,求取极值点位置
        LineImg = cal_image.select(
            ['height', 'layover', 'shadow', 'incAngle', 'x', 'y', 'longitude', 'latitude']).clip(eachLine)
        LineImg_point = LineImg.reduceRegion(
            reducer=ee.Reducer.toList(),
            geometry=cal_image.geometry(),
            scale=30,
            maxPixels=1e13)
        line_points_list.append(LineImg_point)
    list_of_dicts = ee.List(line_points_list).getInfo()  # 需转换数据到本地，严重耗时
    # 分别为右侧被动叠掩点、左侧被动叠掩点和被动阴影点经纬度列表
    r_lon_sum = []
    r_lat_sum = []
    l_lon_sum = []
    l_lat_sum = []
    sh_lon_sum = []
    sh_lat_sum = []
    # 逐视线检索被动畸变
    for PointDict in tqdm(list_of_dicts):
        if orbitProperties_pass == 'ASCENDING':
            order = np.argsort(PointDict['x'])
        elif orbitProperties_pass == 'DESCENDING':
            order = np.argsort(PointDict['x'])[::-1]
        PointDict = {k: np.array(v)[order] for k, v in PointDict.items()}  # 将视线上的像元按照x，从小到大排序
        PointDict['x'], PointDict['y'] = PointDict['x'] * \
                                         10, PointDict['y'] * 10  # 像素行列10m分辨率，由proj得
        EasyIndex = lambda Data, Index, *Keys: [Data[key][Index] for key in Keys]
        # 被动阴影像元识别
        if len(np.where(PointDict['shadow'] == 1)[0]) != 0:
            # 利用grad_fn函数识别出主动阴影起始索引和结束索引
            shadow_grad_fn = lambda shadow_array: [shadow_array[i + 1] - shadow_array[i] for i in
                                                   range(len(shadow_array) - 1)]
            PointDict_shadow_padding = np.insert(PointDict['shadow'], 0, 0)
            shadow_grad = np.array(shadow_grad_fn(PointDict_shadow_padding))
            # 防止PointDict['shadow']最后是以1结尾，导致-1比1个数少一个，且1结尾没有检测被动阴影必要，所以直接去掉
            if PointDict_shadow_padding[-1] == 1:
                shadow_start_index = np.where(shadow_grad == 1)[0][:-1]
            else:
                shadow_start_index = np.where(shadow_grad == 1)[0]
            shadow_range_index = np.where(shadow_grad == -1)[0]
            # 以主动阴影的起始像元位置为参考点
            sh_h, sh_x, s_y, sh_lon, sh_lat, sh_angle = EasyIndex(
                PointDict, shadow_start_index, 'height', 'x', 'y', 'longitude', 'latitude', 'incAngle')
            # 在某一视线中，逐段主动阴影进行分析
            if len(shadow_start_index) != 0:
                for i in range(shadow_start_index.size):
                    start_sh_index = shadow_start_index[i]
                    range_sh_index = shadow_range_index[i]
                    index_Shadow = []
                    # 确定检索像元范围
                    if range_sh_index + 50 < len(PointDict['shadow']):
                        rangeIndex_shadow = range(range_sh_index, range_sh_index + 50)
                    else:
                        rangeIndex_shadow = range(range_sh_index, len(PointDict['shadow']))
                    if len(rangeIndex_shadow) != 0:
                        sh_Range_h, sh_Range_x, sh_Range_y, sh_Range_lon, sh_Range_lat = EasyIndex(
                            PointDict, rangeIndex_shadow, 'height', 'x', 'y', 'longitude', 'latitude')
                        shadow_angle_iRad = np.arctan((sh_h[i] - sh_Range_h) / np.sqrt(
                            np.square(sh_Range_x - sh_x[i]) + np.square(sh_Range_y - s_y[i])))
                        shadow_angle = shadow_angle_iRad * 180 / math.pi
                        index_Shadow = np.where(shadow_angle > (90 - sh_angle[i]))[0]

                    if len(index_Shadow) != 0:
                        tlon_shadow = sh_Range_lon[index_Shadow]
                        tlat_shadow = sh_Range_lat[index_Shadow]
                        for j in range(len(tlon_shadow)):
                            sh_lon_sum.append(tlon_shadow[j])
                            sh_lat_sum.append(tlat_shadow[j])

        # 左右侧被动叠掩
        if len(np.where(PointDict['layover'] == 1)[0]) != 0:
            layover_grad_fn = lambda lay_array: [lay_array[i + 1] - lay_array[i] for i in range(len(lay_array) - 1)]
            PointDict_lay_padding = np.insert(PointDict['layover'], 0, 0)
            # PointDict_lay_padding=np.append(PointDict_lay_padding,0)
            layover_grad = np.array(layover_grad_fn(PointDict_lay_padding))
            if PointDict_lay_padding[-1] == 1:
                rlayover_start_index = np.where(layover_grad == 1)[0][:-1]
            else:
                rlayover_start_index = np.where(layover_grad == 1)[0]
            rlayover_range_index = np.where(layover_grad == -1)[0]
            # 右侧叠掩以rlayover_start_index为参考点，rlayover_range_index为像元检索起点，左侧叠掩相反
            s_h, s_x, s_y, s_lon, s_lat, s_angle = EasyIndex(
                PointDict, rlayover_start_index, 'height', 'x', 'y', 'longitude', 'latitude', 'incAngle')
            d_h, d_x, d_y, d_lon, d_lat, d_angle = EasyIndex(
                PointDict, rlayover_range_index, 'height', 'x', 'y', 'longitude', 'latitude', 'incAngle')
            # 同时进行左右俩侧像元叠掩检测
            for i in range(rlayover_start_index.size):
                start_index = rlayover_start_index[i]
                range_index = rlayover_range_index[i]
                index_Rlayover = []
                index_Llayover = []
                if range_index + 50 < len(PointDict['layover']):
                    rangeIndex = range(range_index, range_index + 50)
                else:
                    rangeIndex = range(range_index, len(PointDict['layover']))

                if start_index - 50 > 0:
                    rangeIndex_l = range(start_index - 50, start_index)
                else:
                    rangeIndex_l = range(start_index)
                # 为检测出完整的被动叠掩位置，像元应同时满足其与主动叠掩起始位置和终止位置的几何关系
                if len(rangeIndex) != 0:
                    r_Range_h, r_Range_x, r_Range_y, r_Range_lon, r_Range_lat = EasyIndex(
                        PointDict, rangeIndex, 'height', 'x', 'y', 'longitude', 'latitude')
                    Rlay_start_iRad = np.arctan(
                        (r_Range_h - s_h[i]) / np.sqrt(np.square(r_Range_x - s_x[i]) + np.square(r_Range_y - s_y[i])))
                    Rlay_end_iRad = np.arctan(
                        (r_Range_h - d_h[i]) / np.sqrt(np.square(r_Range_x - d_x[i]) + np.square(r_Range_y - d_y[i])))
                    Rlay_start = Rlay_start_iRad * 180 / math.pi
                    Rlay_end = Rlay_end_iRad * 180 / math.pi
                    index_Rlayover = np.where(np.logical_and(Rlay_start > s_angle[i], Rlay_end < d_angle[i]))[0]

                if len(rangeIndex_l) != 0:
                    l_Range_h, l_Range_x, l_Range_y, l_Range_lon, l_Range_lat = EasyIndex(
                        PointDict, rangeIndex_l, 'height', 'x', 'y', 'longitude', 'latitude')
                    Llay_start_iRad = np.arctan(
                        (s_h[i] - l_Range_h) / np.sqrt(np.square(s_x[i] - l_Range_x) + np.square(s_y[i] - l_Range_y)))
                    Llay_end_iRad = np.arctan(
                        (d_h[i] - l_Range_h) / np.sqrt(np.square(d_x[i] - l_Range_x) + np.square(d_y[i] - l_Range_y)))
                    Llay_start = Llay_start_iRad * 180 / math.pi
                    Llay_end = Llay_end_iRad * 180 / math.pi
                    index_Llayover = np.where(np.logical_and(Llay_start < s_angle[i], Llay_end > d_angle[i]))[0]

                if len(index_Rlayover) != 0:
                    tlon_RLay = r_Range_lon[index_Rlayover]
                    tlat_RLay = r_Range_lat[index_Rlayover]
                    for j in range(len(tlat_RLay)):
                        r_lon_sum.append(tlon_RLay[j])
                        r_lat_sum.append(tlat_RLay[j])
                if len(index_Llayover) != 0:
                    tlon_LLay = l_Range_lon[index_Llayover]
                    tlat_LLay = l_Range_lat[index_Llayover]
                    for j in range(len(tlat_LLay)):
                        l_lon_sum.append(tlon_LLay[j])
                        l_lat_sum.append(tlat_LLay[j])

    rlay_featurecollection = ee.FeatureCollection(
        [ee.Feature(ee.Geometry.Point(x, y), {'values': 5}) for x, y in zip(r_lon_sum, r_lat_sum)])
    shadow_featurecollection = ee.FeatureCollection(
        [ee.Feature(ee.Geometry.Point(x, y), {'values': 3}) for x, y in zip(sh_lon_sum, sh_lat_sum)])
    llay_featurecollection = ee.FeatureCollection(
        [ee.Feature(ee.Geometry.Point(x, y), {'values': 4}) for x, y in zip(l_lon_sum, l_lat_sum)])
    image_rlayover = ee.Image().paint(rlay_featurecollection, 'values')
    image_llayover = ee.Image().paint(llay_featurecollection, 'values')
    image_shadow = ee.Image().paint(shadow_featurecollection, 'values')
    img_rlayover = image_rlayover.clip(AOI_buffer).reproject(crs=proj, scale=scale)
    img_llayover = image_llayover.clip(AOI_buffer).reproject(crs=proj, scale=scale)
    img_shadow = image_shadow.clip(AOI_buffer).reproject(crs=proj, scale=scale)
    passive_img = ee.ImageCollection([img_rlayover, img_llayover, img_shadow]).mosaic()
    return passive_img
pass_img_out=Line_Correct_test(cal_image,AOI_buffer,Templist, orbitProperties_pass, proj, scale)

100%|███████████████████████████████████████████████████████████████████████████████| 480/480 [00:00<00:00, 713.91it/s]


In [217]:
Map.addLayer(pass_img_out,{'min':3,'max':5,'palette':['00ff00','0000ff','ff0000']},'passive_dis')
Map

Map(bottom=1728391.0, center=[30.17633989163365, 94.28168536055013], controls=(WidgetControl(options=['positio…

In [206]:
# Map2=geemap.Map(basemap='HYBRID')
# Map2.setCenter(94.26, 30.156, 18)
Map.addLayer(llay_featurecollection, {}, 'llay')
Map

Map(bottom=3456861.0, center=[30.16230822258351, 94.26756620407104], controls=(WidgetControl(options=['positio…

In [219]:

image_geometry=ee.Geometry.Rectangle([94.25907322317103, 30.155985535186257,94.3042974980902, 30.196694285644316])
image_rlayover=ee.Image().paint(rlay_featurecollection,'values')
image_llayover=ee.Image().paint(llay_featurecollection,'values')
image_shadow=ee.Image().paint(shadow_featurecollection,'values')
img_rlayover=image_rlayover.clip(image_geometry).reproject(crs=proj, scale=scale)
img_llayover=image_llayover.clip(image_geometry).reproject(crs=proj, scale=scale)
img_shadow=image_shadow.clip(image_geometry).reproject(crs=proj, scale=scale)
passive_img=ee.ImageCollection([img_rlayover,img_llayover,img_shadow]).mosaic()
Map.addLayer(passive_img,{'min':3,'max':5,'palette':['00ff00','0000ff','ff0000']},'passive_dis')
Map

Map(bottom=1728391.0, center=[30.17633989163365, 94.28168536055013], controls=(WidgetControl(options=['positio…

In [132]:
cal_image.geometry().getInfo()

{'geodesic': False,
 'type': 'Polygon',
 'coordinates': [[[94.25907322317103, 30.155985535186257],
   [94.3042974980902, 30.155985535186257],
   [94.3042974980902, 30.196694285644316],
   [94.25907322317103, 30.196694285644316],
   [94.25907322317103, 30.155985535186257]]]}

In [56]:
float_rl=RightLayover.toFloat()
float_rl.getInfo()

{'type': 'Image',
 'bands': [{'id': 'mean',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}

In [113]:

Map2=geemap.Map(basemap='HYBRID')
Map2.setCenter(94.26, 30.156, 18)
# Map2.centerObject(RightLayover, 10)
Map2.addLayer(RightLayover, {}, 'RightLayover')
# Map.addLayer(LeftLayover.randomVisualizer(), {}, 'LeftLayover')
# Map.addLayer(Shadow.randomVisualizer(), {}, 'Shadow')
Map2

Map(center=[30.156, 94.26], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childr…

In [59]:
geemap.ee_export_image(RightLayover,
                    filename='rl_image.tif',
                    scale=10, region=AOI_buffer, file_per_band=False,timeout=1500)

Generating URL ...
Please wait ...
An error occurred while downloading.


In [26]:
RightLayover.getInfo()

{'type': 'Image',
 'bands': [{'id': 'mean',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}